OTTENIMENTO DATA NASCITA ARTISTI - BUON RISULTATO MA PUO ESSERE OTTIMIZZATO

In [2]:
import pandas as pd
import wikipedia
import re
import concurrent.futures
from tqdm import tqdm

wikipedia.set_lang("it")
wikipedia.set_rate_limiting(True)

file_path = "File Unico Sanremo.xlsx"
df = pd.read_excel(file_path, sheet_name="Artisti Unici con genere")

# Filtra solo Maschio e Femmina
df = df[df["Genere"].isin(["Maschio", "Femmina"])].copy()

dob_cache = {}

mesi_ita = {
    "gennaio": "01", "febbraio": "02", "marzo": "03", "aprile": "04",
    "maggio": "05", "giugno": "06", "luglio": "07", "agosto": "08",
    "settembre": "09", "ottobre": "10", "novembre": "11", "dicembre": "12"
}

# Tentativi alternativi per cercare la pagina Wikipedia giusta
def cerca_pagina(artista):
    varianti = [artista]
    parole_chiave = ["cantante", "musicista", "cantautore", "cantautrice", "sanremo"]

    # Aggiungi varianti con parole chiave
    for parola in parole_chiave:
        varianti.append(f"{artista} {parola}")

    for nome in varianti:
        try:
            return wikipedia.page(nome, auto_suggest=False, preload=False)
        except wikipedia.exceptions.DisambiguationError as e:
            continue  # Prova il prossimo
        except wikipedia.exceptions.PageError:
            continue
        except Exception:
            continue

    return None

def estrai_data_nascita(artista):
    if artista in dob_cache:
        return dob_cache[artista]

    if any(sep in artista.lower() for sep in [",", " e ", " - "]):
        print(f"⚠️ Nomi multipli rilevati, salto: {artista}")
        return "Da verificare"

    print(f"🟨 Estrazione data per: {artista}")
    pagina = cerca_pagina(artista)

    if pagina is None:
        print(f"❌ Nessuna pagina trovata per: {artista}")
        return "Da verificare"

    testo = (pagina.summary + " " + pagina.content).lower()[:1000]

    match = re.search(r'(\d{1,2})\s+([a-z]+)\s+(\d{4})', testo)
    if match:
        giorno = match.group(1).zfill(2)
        mese_testo = match.group(2)
        mese = mesi_ita.get(mese_testo)
        anno = match.group(3)

        if mese:
            data = f"{giorno}/{mese}/{anno}"
            dob_cache[artista] = data
            return data

    print(f"⚠️ Nessuna data trovata per: {artista}")
    return "Da verificare"

def apply_parallel(df, func, descrizione):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(func, artista): artista for artista in df["Artista"]}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=descrizione):
            artista = futures[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"❌ Errore durante {descrizione} su: {artista} — {e}")
                results.append("Errore")
    return results

df["Data di nascita"] = apply_parallel(df, estrai_data_nascita, "Estrazione ottimizzata")
output_file = "Artisti_unici_filtro_genere_e_ottimizzati.xlsx"
df.to_excel(output_file, index=False)

print(f"\n✅ File salvato con successo: {output_file}")


🟨 Estrazione data di nascita per: Achille Togliani
🟨 Estrazione data di nascita per: Oscar Carboni
🟨 Estrazione data di nascita per: Gino Latilla
🟨 Estrazione data di nascita per: Zucchero
🟨 Estrazione data di nascita per: Will
🟨 Estrazione data di nascita per: Virginio
🟨 Estrazione data di nascita per: Ultimo
🟨 Estrazione data di nascita per: Tricarico
🟨 Estrazione data di nascita per: Tiziano Orecchio
🟨 Estrazione data di nascita per: Stefano Tosi


Estrazione date:   0%|          | 1/1060 [00:00<09:08,  1.93it/s]

❌ Errore Wikipedia su: Tiziano Orecchio — Page id "Tiziano Orecchio" does not match any pages. Try another id!
🟨 Estrazione data di nascita per: Ugo Molinari


D:\bocci\anaconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file D:\bocci\anaconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Estrazione date:   0%|          | 2/1060 [00:01<10:13,  1.72it/s]

❌ Errore Wikipedia su: Will — "Will" may refer to: 
Will
Will
Will
Will
Will
Will
Brad Will
Will
WiLL
🟨 Estrazione data di nascita per: Gianni Marzocchi
❌ Errore Wikipedia su: Stefano Tosi — "Stefano Tosi" may refer to: 
Stefano Tosi
Stefano Tosi
🟨 Estrazione data di nascita per: Stefano Rosso
❌ Errore Wikipedia su: Ultimo — "Ultimo" may refer to: 
Ultimo
Ultimo
Ultimo
Ultimo
Ultimo
Ultimo - Il capitano che arrestò Totò Riina
Ultimo
Ultimo
Ultimo
Ultimo
Ultimo - La sfida
Ultimo - L'infiltrato
Ultimo - L'occhio del falco
Wikizionario

🟨 Estrazione data di nascita per: Silvano Vittorio


...

Estrazione date:  99%|█████████▉| 1052/1060 [01:23<00:00, 115.58it/s]

⚠️ Nomi multipli rilevati, salto: Mina - Betty Curtis
⚠️ Nomi multipli rilevati, salto: Wilma De Angelis - Gloria Christian
⚠️ Nomi multipli rilevati, salto: Mina - Nelly Fioramonti
⚠️ Nomi multipli rilevati, salto: Mina - Jenny Luna
⚠️ Nomi multipli rilevati, salto: Aura D'Angelo - Silvia Guidi
⚠️ Nomi multipli rilevati, salto: Nadia Liani - Jolanda Rossin
⚠️ Nomi multipli rilevati, salto: Milva - Miriam Del Mare
⚠️ Nomi multipli rilevati, salto: Tonina Torrielli - Nelly Fioramonti
⚠️ Nomi multipli rilevati, salto: Silvia Guidi - Jenny Luna
⚠️ Nomi multipli rilevati, salto: Wanda Romanelli - Nelly Fioramonti
⚠️ Nomi multipli rilevati, salto: Wilma De Angelis - Tanya
⚠️ Nomi multipli rilevati, salto: Wilma De Angelis - Lucia Altieri
⚠️ Nomi multipli rilevati, salto: Nilla Pizzi e Doppio Quintetto Vocale - Teddy Reno e Quartetto Stars
⚠️ Nomi multipli rilevati, salto: Claudio Villa - Eugenia Foligatti
⚠️ Nomi multipli rilevati, salto: Tonina Torrielli - Eugenia Foligatti
⚠️ Nomi multipl

Estrazione date: 100%|██████████| 1060/1060 [01:25<00:00, 12.43it/s] 

⚠️ Nessuna data trovata per: Cugini di Campagna



✅ File salvato con successo: Artisti_unici_con_data_nascita_ottimizzata.xlsx
